In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
#Transfering data to GPU memory will take time and we only do it, if we really need to use GPU
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
cifar10 = keras.datasets.cifar10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [3]:
x_train = x_train.reshape(x_train.shape[0],-1) #reshape 2D image into a 1D vector
x_test = x_test.reshape(x_test.shape[0], -1) #reshape 2D image into a 1D vector
##
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

In [4]:
#normalizing the input vectors
mean = np.mean(x_train)
std = np.std(x_train)
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

In [5]:
no_classes = 10

In [6]:
#change the output to one-hot vectors
y_train = keras.utils.to_categorical(y_train, no_classes)
y_test = keras.utils.to_categorical(y_test, no_classes)

In [7]:
def create_model_without_regularizer(shape_of_input, no_classes, no_hidden_layers=0, no_units_per_layer=50):
    model = Sequential()
    if (no_hidden_layers == 0):
        model.add(Dense(no_classes, input_shape = shape_of_input, name='output_layer', activation='softmax'))
        return model
    model.add(Dense(no_units_per_layer, input_shape = shape_of_input,  activation='relu'))
    for i in range(no_hidden_layers-1):
        model.add(Dense(no_units_per_layer, activation='relu'))
    model.add(Dense(no_classes, name='output_layer', activation='softmax'))
    return model

In [8]:
def create_model_with_regularizer(shape_of_input, no_classes, no_hidden_layers=0, no_units_per_layer=50):
    model = Sequential()
    if (no_hidden_layers == 0):
        model.add(Dense(no_classes, input_shape = shape_of_input, name='output_layer', activation='softmax'))
        return model
    model.add(Dense(no_units_per_layer, input_shape = shape_of_input,  activation='relu'))
    model.add(Dropout(0.15))
    model.add(BatchNormalization())
    for i in range(no_hidden_layers-1):
        model.add(Dense(no_units_per_layer, activation='relu'))
        model.add(Dropout(0.15))
        model.add(BatchNormalization())
    model.add(Dense(no_classes, name='output_layer', activation='softmax'))
    return model

In [13]:
####### model without regularizer
model1 = create_model_without_regularizer((x_train.shape[1],), no_classes, 5, 200)
model1.summary()
model1.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model1.fit(x_train, y_train, epochs=50, batch_size=128, verbose=1)
_, train_accuracy1 = model1.evaluate(x_train, y_train)
train_error1 = (1 - train_accuracy1)*100
_, test_accuracy1 = model1.evaluate(x_test, y_test)
test_error1 = (1 - test_accuracy1)*100
print(f'for model without regularizer: train set error = {train_error1}% and test set error = {test_error1}')
####### model with regularizer
model2 = create_model_with_regularizer((x_train.shape[1],), no_classes, 5, 200)
model2.summary()
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model2.fit(x_train, y_train, epochs=50, batch_size=128, verbose=1)
_, train_accuracy2 = model2.evaluate(x_train, y_train)
train_error2 = (1 - train_accuracy2)*100
_, test_accuracy2 = model2.evaluate(x_test, y_test)
test_error2 = (1 - test_accuracy2)*100
print(f'for model with regularizer: train set error = {train_error2}% and test set error = {test_error2}')
#######

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 200)               614600    
_________________________________________________________________
dense_16 (Dense)             (None, 200)               40200     
_________________________________________________________________
dense_17 (Dense)             (None, 200)               40200     
_________________________________________________________________
dense_18 (Dense)             (None, 200)               40200     
_________________________________________________________________
dense_19 (Dense)             (None, 200)               40200     
_________________________________________________________________
output_layer (Dense)         (None, 10)                2010      
Total params: 777,410
Trainable params: 777,410
Non-trainable params: 0
________________________________________________

Epoch 1/50
391/391 [==============================] - 3s 9ms/step - loss: 1.9702 - accuracy: 0.3083
Epoch 2/50
391/391 [==============================] - 3s 9ms/step - loss: 1.6565 - accuracy: 0.4069
Epoch 3/50
391/391 [==============================] - 3s 9ms/step - loss: 1.5558 - accuracy: 0.4430
Epoch 4/50
391/391 [==============================] - 3s 9ms/step - loss: 1.4835 - accuracy: 0.4696
Epoch 5/50
391/391 [==============================] - 3s 9ms/step - loss: 1.4283 - accuracy: 0.4896
Epoch 6/50
391/391 [==============================] - 3s 8ms/step - loss: 1.3897 - accuracy: 0.5045
Epoch 7/50
391/391 [==============================] - 3s 8ms/step - loss: 1.3544 - accuracy: 0.5178
Epoch 8/50
391/391 [==============================] - 3s 9ms/step - loss: 1.3280 - accuracy: 0.5269
Epoch 9/50
391/391 [==============================] - 3s 9ms/step - loss: 1.2925 - accuracy: 0.5412
Epoch 10/50
391/391 [==============================] - 3s 9ms/step - loss: 1.2642 - accuracy: 0.5498

In [10]:
# plt.xlabel("number of neurons per layer")
# plt.ylabel('test set error %')
# plt.plot(x, y_train_error, 'b', label='train error')
# plt.plot(x, y_test_error, 'r', label='test error')
# plt.legend()

In [11]:
# f = open('number_of_neurons_per_layer___test_set_error_with_regularizer.txt', 'w')

# f.write('number of neurons per layer, train error, test error\n')
# for i in range(len(x)):
#     f.write('{}, {}, {}'.format(x[i], y_train_error[i], y_test_error[i]))
#     f.write('\n')

# f.close()